In [3]:
import openai
from dotenv import load_dotenv
import os
from langchain.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

In [4]:
import gradio as gr

c:\Users\DeLL\Desktop\anything_py\osagyefo_ai\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

C:\Users\DeLL\AppData\Local\Temp\ipykernel_20680\1258215059.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


In [6]:
vector_store = Chroma(
    embedding_function=embedding_function,
    persist_directory="../chroma",
    collection_name="osagyefo_v1"
)

C:\Users\DeLL\AppData\Local\Temp\ipykernel_20680\1161908860.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


In [7]:
retriever = vector_store.as_retriever(
    search_type="mmr",  # Maximal Marginal Relevance
    search_kwargs={"k": 5, "lambda_mult": 0.5}
)

In [8]:
print(vector_store._collection.count())  

1323


In [9]:
retriever.get_relevant_documents("my mother did not give me food")

C:\Users\DeLL\AppData\Local\Temp\ipykernel_20680\3153153935.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("my mother did not give me food")


[Document(metadata={'doc_type': 'act', 'total_pages': 115, 'chapter_title': 'CHAPTER EIGHT', 'file_name': 'criminal offences act.pdf'}, page_content='CHAPTER EIGHT\nPublic Nuisance\nHindering Burials\n285.   Hindering burial of dead body\nA person who unlawfully hinders the burial of the d ead body of a person, or without lawful authority\ndisinters, dissects or harms the dead body of a person, or being under a duty to cause the dead body of a\nperson to be buried, fails to perform that duty commits a misdemeanour.\nUnwholesome Food\n286.   Selling unwholesome food\nA person who sells, or prepares or offers for sale, as being fit for consumption as food or drink, a thing\nwhich that person knows or has reason to believe that it is in a condition of putrefaction, adulteration, or\nother cause, as to be likely to be noxious to health, commits a misdemeanour.\nNoxious Trade\n287.   Carrying on of noxious trade, interference with public rights\nA person who, without lawful authority or exc

In [10]:
load_dotenv(override=True)
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
TOGETHER_API_KEY = os.getenv("TOGETHER_API_KEY")

model_1 = "gemma2-9b-it"
client1 = openai.OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ.get("GROQ_API_KEY"),
)

model_2 = "meta-llama/Llama-3.3-70B-Instruct-Turbo-Free"
client2 = openai.OpenAI(
  api_key=os.environ.get("TOGETHER_API_KEY"),
  base_url="https://api.together.xyz/v1",
)

Model tests

In [13]:

def rephrase_query(user_query):
    pdf_path = "../docs"
    prompt_path = "../prompts/rephaser_llm_template.txt"
    
    with open(prompt_path, "r", encoding="utf-8") as f:
        system_prompt = f.read().strip()
        system_prompt += "Your knowledge is limited strictly to the content of the following files:"
        for filename in os.listdir(pdf_path):
            system_prompt += filename + "\n"


    message = [{'role': "system", "content": system_prompt}, {'role': "system", "content": user_query}]
    rephrased_query = client1.chat.completions.create(model=model_1, messages=message)
    rephrased_query = rephrased_query.choices[0].message.content

    return rephrased_query

In [16]:
rephrase_query("I want to divorce my wife")

"['Seek legal dissolution of marriage', 'Requirements for divorce in Ghana', 'How to legally end a marriage in Ghana'] \n"

In [ ]:

def rephrase_query(user_query):
    pdf_path = "../docs"
    prompt_path = "../prompts/rephaser_llm_template.txt"
    
    with open(prompt_path, "r", encoding="utf-8") as f:
        system_prompt = f.read().strip()
        system_prompt += "Your knowledge is limited strictly to the content of the following files:"
        for filename in os.listdir(pdf_path):
            system_prompt += filename + "\n"


    message = [{'role': "system", "content": system_prompt}, {'role': "system", "content": user_query}]
    rephrased_query = client2.chat.completions.create(model=model_2, messages=message)
    rephrased_query = rephrased_query.choices[0].message.content

    return rephrased_query

PIPELINE

In [17]:
"""
LLM 1 will rephrase every question into 3 different similar questions

"""

def rephrase_query(user_query):
    pdf_path = "../docs"
    prompt_path = "../prompts/rephaser_llm_template.txt"
    
    with open(prompt_path, "r", encoding="utf-8") as f:
        system_prompt = f.read().strip()
        system_prompt += "Your knowledge is limited strictly to the content of the following files:"
        for filename in os.listdir(pdf_path):
            system_prompt += filename + "\n"


    message = [{'role': "system", "content": system_prompt}, {'role': "system", "content": user_query}]
    rephrased_query = client1.chat.completions.create(model=model_1, messages=message)
    rephrased_query = rephrased_query.choices[0].message.content

    return rephrased_query

In [21]:
rephrased_query = rephrase_query("i mistakenly hit someone with my car")
print(rephrased_query)

context = "" 
relevant_docs = retriever.get_relevant_documents(rephrased_query)

for doc in relevant_docs:
    context += f"[Source: {doc.metadata['file_name']}, Chapter: {doc.metadata.get('chapter_title', 'N/A')}]\n{doc.page_content}\n\n"

['What are the legal consequences if a person unintentionally collides with another individual while driving?', 'Can I be charged with an offense if I accidentally struck someone with my vehicle?', 'What are the applicable laws in Ghana regarding vehicular accidents resulting in unintentional harm?'] 



In [22]:
def chat_osagyefo(user_query, history):
    
    prompt_path = "../prompts/main_prompt_template.txt"

    rephrased_query = rephrase_query(user_query)
    relevant_docs = retriever.get_relevant_documents(rephrased_query)

    print("RELEVANT DOCS: ", relevant_docs)
    context = ''
    for doc in relevant_docs:
        context += f"[Source: {doc.metadata['file_name']}, Chapter: {doc.metadata.get('chapter_title', 'N/A')}]\n{doc.page_content}\n\n"

    # loading the prompt
    with open(prompt_path, "r", encoding="utf-8") as f:
        osagyefo_system_prompt = f.read().strip()
    osagyefo_system_prompt += f"Use the following context to answer:\n{context}"



    history = [
        {'role': h['role'], 'content': h['content'] }
        for h in history
        if 'role' in h and 'content' in h
    ]

    msg_to_model = [{"role": "system", "content": osagyefo_system_prompt}] + history + [{"role": "user", "content": user_query}]

    model = client2.chat.completions.create(
        model=model_2,
        messages=msg_to_model,
    )

    osagyefo_response = model.choices[0].message.content
    
    return osagyefo_response

In [23]:
demo_chatbot = gr.ChatInterface(chat_osagyefo,
                                title="OSAGYEFO ESQ.",
                                description="Welcome! Put forth your legal challenges. ")

c:\Users\DeLL\Desktop\anything_py\osagyefo_ai\.venv\lib\site-packages\gradio\chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


In [24]:
demo_chatbot.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


RELEVANT DOCS:  [Document(metadata={'chapter_title': 'Chapter IV', 'total_pages': 161, 'file_name': 'ghana_constitution.pdf', 'doc_type': 'constitution'}, page_content='Chapter IV\n-The Laws of Ghana'), Document(metadata={'chapter_title': 'Chapter but', 'total_pages': 161, 'file_name': 'ghana_constitution.pdf', 'doc_type': 'constitution'}, page_content='tribunals for the trial of offences against military law committed by persons subject to military \nlaw. \n(20) Where a person subject to military law, who is not in active service, commits an offence \nwhich is within the jurisdiction of a civil court, he shall not be tried by a court-martial or \nmilitary tribunal for the offence unless the offence is within the jurisdiction of a court-martial \nor other military tribunal under any law for the enforcement of military discipline. \n(21) For the purposes of this article, “criminal offence” means a criminal offence under the \nlaws of Ghana. \n20. PROTECTION FROM DEPRIVATION OF PROPERTY.